## Importing the data

In [1]:
import pandas as pd
import pm4py
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("event_logs/filtered_event_log_complete_clear_invoice.csv", sep=";", parse_dates=["event_timestamp"])

The aim of this project is to use case attributes of the purchase orders to create a model that attempts to predict the throughput time of each purchase order. To achieve this, an appropriate dataset must be created from the filtered event log.

## Creating a table of case attributes for each line item

The PR_NPR, product types, vendor and value are generally consistent across all of the events related to a particular line item.  
The associated resources and occurrence of different events in each line items trace will need to be added after.

In [3]:
data.groupby("case_id").nunique().mean()

case_PO_id               1.000000
event_id                 5.652626
event_name               5.517371
event_timestamp          5.607721
event_resource           5.432430
event_value_EUR          1.001507
case_vendor              1.000000
case_PR_NPR              0.990687
case_product_type        0.990687
case_product_specific    0.990687
dtype: float64

Starting with the beginning event time stamp and case attributes:

In [4]:
start_case = data.groupby("case_id").nth(0)[["event_timestamp", "event_value_EUR", "case_vendor", "case_PR_NPR", "case_product_type", "case_product_specific"]]
start_case.rename(columns={"event_timestamp":"start_timestamp"})
start_case.head()

,event_timestamp,event_value_EUR,case_vendor,case_PR_NPR,case_product_type,case_product_specific
case_id,,,,,,
4507000266_00010,2018-02-01 08:53:00+00:00,134.0,vendorID_0103,PR,Packaging,Labels
4507000268_00010,2018-02-01 08:56:00+00:00,212.0,vendorID_0105,PR,Packaging,Labels
4507000268_00020,2018-02-01 08:56:00+00:00,389.0,vendorID_0105,PR,Packaging,Labels
4507000272_00010,2018-02-01 09:06:00+00:00,266.0,vendorID_0137,NPR,Sales,Products for Resale
4507000272_00040,2018-02-01 09:06:00+00:00,1272.0,vendorID_0137,NPR,Sales,Products for Resale


Adding the end event timestamp:

In [5]:
end_case = data.groupby("case_id").nth(-1)["event_timestamp"]
end_case = end_case.rename("end_timestamp")

Calculating the case total throughput for each line item:

In [6]:
li_data = pd.merge(start_case, end_case, left_index=True, right_index=True, how="outer")
li_data = li_data.rename(columns={"event_timestamp":"start_timestamp"})

In [7]:
li_data["throughput"] = li_data["end_timestamp"] - li_data["start_timestamp"]

In [8]:
li_data.head()

,start_timestamp,event_value_EUR,case_vendor,case_PR_NPR,case_product_type,case_product_specific,end_timestamp,throughput
case_id,,,,,,,,
4507000266_00010,2018-02-01 08:53:00+00:00,134.0,vendorID_0103,PR,Packaging,Labels,2018-04-26 09:14:00+00:00,84 days 00:21:00
4507000268_00010,2018-02-01 08:56:00+00:00,212.0,vendorID_0105,PR,Packaging,Labels,2018-09-05 12:10:00+00:00,216 days 03:14:00
4507000268_00020,2018-02-01 08:56:00+00:00,389.0,vendorID_0105,PR,Packaging,Labels,2018-09-05 12:10:00+00:00,216 days 03:14:00
4507000272_00010,2018-02-01 09:06:00+00:00,266.0,vendorID_0137,NPR,Sales,Products for Resale,2018-09-05 12:12:00+00:00,216 days 03:06:00
4507000272_00040,2018-02-01 09:06:00+00:00,1272.0,vendorID_0137,NPR,Sales,Products for Resale,2018-09-05 12:12:00+00:00,216 days 03:06:00


Some items have no product types included:

In [9]:
li_data[li_data["case_PR_NPR"].isna()].head(5)

,start_timestamp,event_value_EUR,case_vendor,case_PR_NPR,case_product_type,case_product_specific,end_timestamp,throughput
case_id,,,,,,,,
4507002823_00030,2018-01-15 08:40:00+00:00,208.0,vendorID_0103,NaN,NaN,NaN,2018-04-26 09:14:00+00:00,101 days 00:34:00
4507002823_00050,2018-01-15 08:40:00+00:00,338.0,vendorID_0103,NaN,NaN,NaN,2018-04-26 09:14:00+00:00,101 days 00:34:00
4507002823_00060,2018-01-15 08:40:00+00:00,507.0,vendorID_0103,NaN,NaN,NaN,2018-04-26 09:14:00+00:00,101 days 00:34:00
4507002823_00070,2018-01-15 08:40:00+00:00,338.0,vendorID_0103,NaN,NaN,NaN,2018-04-26 09:14:00+00:00,101 days 00:34:00
4507002823_00080,2018-01-15 08:40:00+00:00,338.0,vendorID_0103,NaN,NaN,NaN,2018-04-26 09:14:00+00:00,101 days 00:34:00


In [10]:
li_data = li_data.dropna(how="any")

### Feature engineering other relevant case attributes

Finding rework rate per case:

In [11]:
rework_activities = ['Change Price', 'Change Quantity', 'Change Delivery Indicator',
                     'Vendor creates debit memo', 'Cancel Invoice Receipt','Change Storage Location', 
                     'Change Currency', 'Reactivate Purchase Order Item', 'Change Approval for Purchase Order', 
                     'Cancel Goods Receipt','Cancel Subsequent Invoice', 'Update Order Confirmation', 
                     'Record Subsequent Invoice', 'Change payment term']

In [12]:
li_data = pd.merge(li_data, data[data["event_name"].isin(rework_activities)].groupby("case_id").count()["event_name"].rename("rework_activities"), 
         how="left", left_index=True, right_index=True)
li_data["rework_activities"] = li_data["rework_activities"].fillna(0)

Was a payment block used? ("Remove payment block" occurred):

In [13]:
cases_w_pb = pd.DataFrame(data[data["event_name"]=="Remove Payment Block"]["case_id"].unique())
cases_w_pb["payment_block"] = True
cases_w_pb = cases_w_pb.set_index(0)

In [14]:
li_data = pd.merge(li_data, cases_w_pb, how="left", left_index=True, right_index=True)
li_data["payment_block"] = li_data["payment_block"].fillna(False)

Automation rate per case:

In [15]:
data["automation"] = data["event_resource"].str[0]
data["automation"] = data["automation"].replace({"u": False, 
                                                 "b": True, 
                                                 "N" : np.nan})

In [16]:
li_data = pd.merge(li_data, data.dropna().groupby("case_id")["automation"].mean()*100, how="left", left_index=True, right_index=True)

Average daily workload (number of cases being worked on) for the duration of the case, per vendor:

In [17]:
data["event_date"] = data["event_timestamp"].dt.date

In [18]:
case_avg_vendor_workload = pd.read_csv("throughput_dataset/case_avg_vendor_workload.csv").set_index("case_id")

In [19]:
li_data = pd.merge(li_data, case_avg_vendor_workload, how="left", left_index=True, right_index=True)

Finished line item level case dataset:

In [20]:
li_data = li_data.reset_index()

In [21]:
li_data.head()

,case_id,start_timestamp,event_value_EUR,case_vendor,case_PR_NPR,case_product_type,case_product_specific,end_timestamp,throughput,rework_activities,payment_block,automation,avg_vendor_workload
0,4507000266_00010,2018-02-01 08:53:00+00:00,134.0,vendorID_0103,PR,Packaging,Labels,2018-04-26 09:14:00+00:00,84 days 00:21:00,0.0,False,0.0,49.272727
1,4507000268_00010,2018-02-01 08:56:00+00:00,212.0,vendorID_0105,PR,Packaging,Labels,2018-09-05 12:10:00+00:00,216 days 03:14:00,0.0,False,0.0,13.402778
2,4507000268_00020,2018-02-01 08:56:00+00:00,389.0,vendorID_0105,PR,Packaging,Labels,2018-09-05 12:10:00+00:00,216 days 03:14:00,0.0,False,0.0,13.402778
3,4507000272_00010,2018-02-01 09:06:00+00:00,266.0,vendorID_0137,NPR,Sales,Products for Resale,2018-09-05 12:12:00+00:00,216 days 03:06:00,0.0,False,25.0,3.647059
4,4507000272_00040,2018-02-01 09:06:00+00:00,1272.0,vendorID_0137,NPR,Sales,Products for Resale,2018-09-05 12:12:00+00:00,216 days 03:06:00,0.0,False,25.0,3.647059


## Creating a table of case attributes for each purchase order

Many of the attributes in the line item dataset are taken from the purchase order level, and thus duplicated across multiple observations.  
It therefore may be viable to group cases by the purchase order instead of line item.

Are all of the line items in one purchase always cleared ("clear invoice") on the same day? The majority yes (6961) but 6.5% are not (486).

In [22]:
pd.DataFrame(data.groupby("case_PO_id").apply(lambda group: group[group["event_name"]=="Clear Invoice"])["event_timestamp"]).reset_index().groupby("case_PO_id").nunique()["event_timestamp"].value_counts()

1    6961
2     418
3      55
4      11
7       1
5       1
Name: event_timestamp, dtype: int64

Within one purchase order document, there is on average 3.5 different line items.   

Attributes that are the same for every line item in each purchase order:
* vendor
* PR_NPR
* product type
* specific product type  
* The process (trace) followed seems to be the same for every line item in one purchase order, with the same resources working on each of the items.

Attributes that differ for every line item in each purchase order, and will need to be averaged:
* line item values

Attributes that will need to be calculated from one of the line items in the purchase order:
* automation rate
* rework rate
* payment block occurred

Attributes that only exist at the purchase order level:
* number of line items in one purchase order 

In [23]:
data.groupby("case_PO_id").nunique().mean()

case_id                   3.474822
event_id                 19.641869
event_name                5.707265
event_timestamp           7.268967
event_resource            5.732241
event_value_EUR           3.113737
case_vendor               1.000000
case_PR_NPR               0.997449
case_product_type         1.007923
case_product_specific     1.035853
automation                1.283201
event_date                5.422452
dtype: float64

Creating a case attributes dataframe, from the purchase order level:

Because the cases within a purchase order tend to follow the same process, be worked on by the same resources, etc, one line item case is used as the "representative" of the line items in a purchase order.

In [24]:
li_data["case_PO_id"] = li_data["case_id"].str.split("_", expand=True)[0]

In [25]:
PO_data = li_data.groupby("case_PO_id").nth(0).reset_index()

In [26]:
PO_data.head()

,case_PO_id,case_id,start_timestamp,event_value_EUR,case_vendor,case_PR_NPR,case_product_type,case_product_specific,end_timestamp,throughput,rework_activities,payment_block,automation,avg_vendor_workload
0,4507000266,4507000266_00010,2018-02-01 08:53:00+00:00,134.0,vendorID_0103,PR,Packaging,Labels,2018-04-26 09:14:00+00:00,84 days 00:21:00,0.0,False,0.0,49.272727
1,4507000268,4507000268_00010,2018-02-01 08:56:00+00:00,212.0,vendorID_0105,PR,Packaging,Labels,2018-09-05 12:10:00+00:00,216 days 03:14:00,0.0,False,0.0,13.402778
2,4507000272,4507000272_00010,2018-02-01 09:06:00+00:00,266.0,vendorID_0137,NPR,Sales,Products for Resale,2018-09-05 12:12:00+00:00,216 days 03:06:00,0.0,False,25.0,3.647059
3,4507000273,4507000273_00010,2018-02-01 09:25:00+00:00,528.0,vendorID_0138,NPR,Sales,Products for Resale,2018-08-03 11:21:00+00:00,183 days 01:56:00,0.0,True,0.0,9.121951
4,4507000287,4507000287_00010,2018-02-01 10:17:00+00:00,583.0,vendorID_0148,NPR,Sales,Products for Resale,2018-08-02 12:43:00+00:00,182 days 02:26:00,0.0,True,0.0,3.714286


### Feature engineering purchase order level case attributes

In [27]:
PO_data = PO_data.set_index("case_PO_id")

Number of line items in each purchase order:

In [28]:
num_items = data.groupby("case_PO_id").nunique()["case_id"].rename("num_items")
num_items.index = num_items.index.astype("str")

In [29]:
PO_data = pd.merge(PO_data, num_items, how="left", left_index=True, right_index=True)

Average line item value in each purchase order:

In [30]:
avg_value = data.groupby("case_PO_id").apply(lambda group: group[group["event_name"]=="Create Purchase Order Item"])["event_value_EUR"].reset_index().groupby("case_PO_id").mean()["event_value_EUR"]
avg_value.index = avg_value.index.astype("str")

In [31]:
PO_data["event_value_EUR"] = avg_value
PO_data = PO_data.rename(columns={"event_value_EUR":"avg_value_EUR"})

Final purchase order level dataset:

In [32]:
PO_data = PO_data.reset_index()

In [33]:
PO_data.head()

,case_PO_id,case_id,start_timestamp,avg_value_EUR,case_vendor,case_PR_NPR,case_product_type,case_product_specific,end_timestamp,throughput,rework_activities,payment_block,automation,avg_vendor_workload,num_items
0,4507000266,4507000266_00010,2018-02-01 08:53:00+00:00,134.0,vendorID_0103,PR,Packaging,Labels,2018-04-26 09:14:00+00:00,84 days 00:21:00,0.0,False,0.0,49.272727,1
1,4507000268,4507000268_00010,2018-02-01 08:56:00+00:00,300.5,vendorID_0105,PR,Packaging,Labels,2018-09-05 12:10:00+00:00,216 days 03:14:00,0.0,False,0.0,13.402778,2
2,4507000272,4507000272_00010,2018-02-01 09:06:00+00:00,769.0,vendorID_0137,NPR,Sales,Products for Resale,2018-09-05 12:12:00+00:00,216 days 03:06:00,0.0,False,25.0,3.647059,2
3,4507000273,4507000273_00010,2018-02-01 09:25:00+00:00,598.5,vendorID_0138,NPR,Sales,Products for Resale,2018-08-03 11:21:00+00:00,183 days 01:56:00,0.0,True,0.0,9.121951,2
4,4507000287,4507000287_00010,2018-02-01 10:17:00+00:00,581.5,vendorID_0148,NPR,Sales,Products for Resale,2018-08-02 12:43:00+00:00,182 days 02:26:00,0.0,True,0.0,3.714286,2


## Pre-processing for modelling

For the later modelling, having too many one-hot encoded features created from the factor levels could cause dimensionality problems.   
Factor level aggregation is explored in the notebook "Exploring the Processed Dataset", and implemented here.

The datasets also need to be formatted so that they can be processed by an algorithm: e.g. dropping non-numerical id columns, etc.

In [34]:
li_X = li_data[["event_value_EUR", "case_vendor", "case_PR_NPR", "case_product_type", "case_product_specific", "rework_activities", "payment_block", "automation", "avg_vendor_workload"]]
li_y = li_data["throughput"]

In [35]:
PO_X = PO_data[["avg_value_EUR", "case_vendor", "case_PR_NPR", "case_product_type", "case_product_specific", "rework_activities", "payment_block", "automation", "avg_vendor_workload", "num_items"]]          
PO_y = PO_data["throughput"]

In [36]:
li_y = li_y.dt.days
PO_y = PO_y.dt.days

"other" levels always start with _ so that it becomes the reference group in the models.

In [37]:
# PR/NPR
li_X = li_X.replace({"case_PR_NPR" : {"NPR":"_NPR", "OTHER":"_NPR"}})

In [38]:
# group product types
product_types = {key: "_other_product_type" for key in li_data["case_product_type"].unique()}
product_types["Packaging"] = "Packaging"
product_types["Sales"] = "Sales"
product_types["Additives"] = "Additives"
li_X = li_X.replace({"case_product_type" : product_types})

In [39]:
# specific product types
specific_types = {key: "_other_specific_type" for key in li_data["case_product_specific"].unique()}
specific_types["Products for Resale"] = "Products for Resale"
specific_types["Labels"] = "Labels"
specific_types["Metal Containers & Lids < 30L"] = "Containers"
specific_types["Plastic Containers & Lids < 30L"] = "Containers"
specific_types["Extenders"] = "Extenders"
li_X = li_X.replace({"case_product_specific" : specific_types})

In [40]:
# vendors
vendors = {key: "_other_vendor" for key in li_data["case_vendor"].unique()}
vendors["vendorID_0120"] = "vendorID_0120"
vendors["vendorID_0104"] = "vendorID_0104"
vendors["vendorID_0171"] = "vendorID_0171"
vendors["vendorID_0136"] = "vendorID_0136"
vendors["vendorID_0106"] = "vendorID_0106"
li_X = li_X.replace({"case_vendor" : vendors})

In [41]:
li_X = li_X.replace({"payment_block" : {True:1, False:0}})

In [42]:
li_X

,event_value_EUR,case_vendor,case_PR_NPR,case_product_type,case_product_specific,rework_activities,payment_block,automation,avg_vendor_workload
0,134.0,_other_vendor,PR,Packaging,Labels,0.0,0,0.0,49.272727
1,212.0,_other_vendor,PR,Packaging,Labels,0.0,0,0.0,13.402778
2,389.0,_other_vendor,PR,Packaging,Labels,0.0,0,0.0,13.402778
3,266.0,_other_vendor,_NPR,Sales,Products for Resale,0.0,0,25.0,3.647059
4,1272.0,_other_vendor,_NPR,Sales,Products for Resale,0.0,0,25.0,3.647059
...,...,...,...,...,...,...,...,...,...
25631,142.0,_other_vendor,_NPR,Sales,Products for Resale,0.0,1,20.0,14.726027
25632,44.0,_other_vendor,_NPR,Sales,Products for Resale,0.0,1,20.0,14.726027
25633,14.0,_other_vendor,_NPR,Sales,Products for Resale,0.0,1,20.0,14.726027
25634,27.0,_other_vendor,_NPR,Sales,Products for Resale,0.0,1,20.0,14.726027


In [43]:
# PR/NPR
PO_X = PO_X.replace({"case_PR_NPR" : {"NPR":"_NPR", "OTHER":"_NPR"}})

In [44]:
# group product types
product_types = {key: "_other_product_type" for key in PO_data["case_product_type"].unique()}
product_types["Packaging"] = "Packaging"
product_types["Sales"] = "Sales"
product_types["Additives"] = "Additives"
PO_X = PO_X.replace({"case_product_type" : product_types})

In [45]:
# specific product types
specific_types = {key: "_other_specific_type" for key in PO_data["case_product_specific"].unique()}
specific_types["Products for Resale"] = "Products for Resale"
specific_types["Labels"] = "Labels"
specific_types["Metal Containers & Lids < 30L"] = "Containers"
specific_types["Plastic Containers & Lids < 30L"] = "Containers"
specific_types["Extenders"] = "Extenders"
PO_X = PO_X.replace({"case_product_specific" : specific_types})

In [46]:
# vendors
vendors = {key: "_other_vendor" for key in PO_data["case_vendor"].unique()}
vendors["vendorID_0120"] = "vendorID_0120"
vendors["vendorID_0104"] = "vendorID_0104"
vendors["vendorID_0171"] = "vendorID_0171"
vendors["vendorID_0136"] = "vendorID_0136"
vendors["vendorID_0106"] = "vendorID_0106"
PO_X = PO_X.replace({"case_vendor" : vendors})

In [47]:
PO_X = PO_X.replace({"payment_block" : {True:1, False:0}})

Comparing the full versus new modelling dataset:

In [48]:
PO_model_data = pd.concat([PO_X, PO_y], axis=1)
li_model_data = pd.concat([li_X, li_y], axis=1)

In [49]:
PO_data

,case_PO_id,case_id,start_timestamp,avg_value_EUR,case_vendor,case_PR_NPR,case_product_type,case_product_specific,end_timestamp,throughput,rework_activities,payment_block,automation,avg_vendor_workload,num_items
0,4507000266,4507000266_00010,2018-02-01 08:53:00+00:00,134.000000,vendorID_0103,PR,Packaging,Labels,2018-04-26 09:14:00+00:00,84 days 00:21:00,0.0,False,0.0,49.272727,1
1,4507000268,4507000268_00010,2018-02-01 08:56:00+00:00,300.500000,vendorID_0105,PR,Packaging,Labels,2018-09-05 12:10:00+00:00,216 days 03:14:00,0.0,False,0.0,13.402778,2
2,4507000272,4507000272_00010,2018-02-01 09:06:00+00:00,769.000000,vendorID_0137,NPR,Sales,Products for Resale,2018-09-05 12:12:00+00:00,216 days 03:06:00,0.0,False,25.0,3.647059,2
3,4507000273,4507000273_00010,2018-02-01 09:25:00+00:00,598.500000,vendorID_0138,NPR,Sales,Products for Resale,2018-08-03 11:21:00+00:00,183 days 01:56:00,0.0,True,0.0,9.121951,2
4,4507000287,4507000287_00010,2018-02-01 10:17:00+00:00,581.500000,vendorID_0148,NPR,Sales,Products for Resale,2018-08-02 12:43:00+00:00,182 days 02:26:00,0.0,True,0.0,3.714286,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7318,4508046815,4508046815_00010,2018-02-08 06:33:00+00:00,3556.777778,vendorID_0122,NPR,CAPEX & SOCS,Laboratory Supplies & Services,2018-08-23 11:39:00+00:00,196 days 05:06:00,0.0,False,0.0,18.709302,9
7319,4508046831,4508046831_00010,2018-02-08 07:18:00+00:00,1053.666667,vendorID_0382,NPR,Sales,Products for Resale,2018-08-23 11:40:00+00:00,196 days 04:22:00,0.0,True,20.0,27.800000,3
7320,4508046840,4508046840_00010,2018-02-08 07:42:00+00:00,201.666667,vendorID_0477,NPR,Sales,Products for Resale,2018-08-23 11:39:00+00:00,196 days 03:57:00,0.0,True,20.0,2.083333,3
7321,4508047026,4508047026_00010,2018-03-08 06:39:00+00:00,157.714286,vendorID_0114,NPR,Sales,Products for Resale,2018-08-23 11:39:00+00:00,168 days 05:00:00,0.0,True,20.0,14.726027,7


In [50]:
PO_model_data

,avg_value_EUR,case_vendor,case_PR_NPR,case_product_type,case_product_specific,rework_activities,payment_block,automation,avg_vendor_workload,num_items,throughput
0,134.000000,_other_vendor,PR,Packaging,Labels,0.0,0,0.0,49.272727,1,84
1,300.500000,_other_vendor,PR,Packaging,Labels,0.0,0,0.0,13.402778,2,216
2,769.000000,_other_vendor,_NPR,Sales,Products for Resale,0.0,0,25.0,3.647059,2,216
3,598.500000,_other_vendor,_NPR,Sales,Products for Resale,0.0,1,0.0,9.121951,2,183
4,581.500000,_other_vendor,_NPR,Sales,Products for Resale,0.0,1,0.0,3.714286,2,182
...,...,...,...,...,...,...,...,...,...,...,...
7318,3556.777778,_other_vendor,_NPR,_other_product_type,_other_specific_type,0.0,0,0.0,18.709302,9,196
7319,1053.666667,_other_vendor,_NPR,Sales,Products for Resale,0.0,1,20.0,27.800000,3,196
7320,201.666667,_other_vendor,_NPR,Sales,Products for Resale,0.0,1,20.0,2.083333,3,196
7321,157.714286,_other_vendor,_NPR,Sales,Products for Resale,0.0,1,20.0,14.726027,7,168
